In [1]:
import numpy as np
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
import scanpy as sc
import scipy

In [2]:
adata = ad.read_h5ad("/Users/apple/Desktop/KB/data/BiddyData/biddy_10215_2000.h5ad")

In [5]:
adata_larry = ad.read_h5ad("/Users/apple/Desktop/KB/data/LarryData/Larry_41093_2000_norm_log.h5ad")

In [7]:
adata_larry.obs["clone_id"].value_counts().median()

10.0

In [8]:
adata.obs["clone_id"].value_counts().median()

9.0

In [3]:
adata.shape

(10215, 2000)

In [4]:
adata.obs

,timecourse,reprogramming_day,reprogramming,cell_type,cell_cycle,cluster,monocle_state,pseudotime,CellTagD0_85k,CellTagD3_85k,CellTagD13_85k,CellTagD0_48k,CellTagD3_48k,CellTagD13_48k,clone_id
HF1_AAAGCAATCCAGATCA_5,1,12,NaN,Ambiguous,G1,5,2,0.426868,424.0,476.0,NaN,424.0,476.0,NaN,424.0
HF1_AACACGTAGAACAACT_5,1,12,NaN,Ambiguous,S,5,2,0.549377,195.0,NaN,NaN,195.0,NaN,NaN,195.0
HF1_AAATGCCTCTAACTGG_5,1,12,NaN,NaN,G1,0,NaN,NaN,NaN,478.0,NaN,493.0,478.0,NaN,493.0
HF1_AAACCTGAGAGCAATT_5,1,12,NaN,Fibroblast,G2M,2,2,0.664971,647.0,309.0,NaN,647.0,309.0,NaN,647.0
HF1_AACACGTTCGAGGTAG_5,1,12,False,iEP,G1,2,2,0.730338,487.0,240.0,NaN,487.0,240.0,NaN,487.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HF2_TTCCCAGGTGATGATA_4,2,9,NaN,iEP,G2M,5,2,0.530828,2367.0,2514.0,NaN,2367.0,2514.0,NaN,2367.0
HF2_TTCTCCTCATGAACCT_4,2,9,NaN,iEP,G2M,5,2,0.403314,2490.0,2042.0,NaN,2490.0,2042.0,NaN,2490.0
HF2_TTCTACAAGATAGCAT_4,2,9,NaN,iEP,G2M,5,2,1.702257,2815.0,NaN,NaN,2815.0,NaN,NaN,2815.0
HF2_TTGGAACCAAGAAAGG_4,2,9,NaN,Ambiguous,G1,2,2,1.029464,2764.0,2525.0,NaN,2764.0,2525.0,NaN,2764.0


In [5]:
# Step 1: Identify unique clone_ids and their frequencies
clone_id_counts = adata.obs['clone_id'].value_counts()

In [6]:
# Step 2: Initialize empty lists to hold indices for adata_train and adata_test
test_indices = []
train_indices = []

# Step 3: Loop through each clone_id and split based on the criteria
for clone_id, count in clone_id_counts.items():
    clone_indices = adata.obs[adata.obs['clone_id'] == clone_id].index
    
    if count >= 10:
        # Select 10% of the elements randomly for adata_test
        test_size = int(np.ceil(0.1 * count))
        test_clone_indices = np.random.choice(clone_indices, size=test_size, replace=False)
        test_indices.extend(test_clone_indices)
        
        # The remaining elements go to adata_train
        train_clone_indices = list(set(clone_indices) - set(test_clone_indices))
        train_indices.extend(train_clone_indices)
    else:
        # All elements go to adata_train
        train_indices.extend(clone_indices)



In [7]:

# Step 4: Subset the original adata to create adata_train and adata_test
adata_train = adata[train_indices, :].copy()
adata_test = adata[test_indices, :].copy()


In [8]:
adata_test.shape, adata_train.shape

((995, 2000), (9220, 2000))

In [10]:
adata_test.write_h5ad('Biddy_test.h5ad')
adata_train.write_h5ad('Biddy_train.h5ad')

